In [1]:
## using traditional network analysis as a comparision for our method.
import os
import glob
import pandas as pd
import numpy as np
import re
import networkx as nx
import networkx.algorithms.community as nx_comm
import matplotlib.pyplot as plt

input_dir = '/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix?'
output_dir = 'adni_out03_network_analysis'

input_files = glob.glob(input_dir + '/*.txt')
input_files.sort()
# input_files

In [2]:
# input_files[:10]
subject_id = []
ses_id = []

for f in input_files:
    subjid = re.search('(.*)_sub-(.*)-ses-(.*).txt', f).group(2)
    subject_id.append(subjid)
    sesid = re.search('(.*)_sub-(.*)-ses-(.*).txt', f).group(3)
    ses_id.append(sesid)


In [6]:

output_file = pd.DataFrame({'subject_id': subject_id, 'ses_id': ses_id})

all_clustering = []
all_betweenness = []
all_modularity = []
all_degree = []
all_eigenvector = []
all_closeness = []

for f in input_files:
    print(f)
    df_corr = pd.read_csv(f)
    
    # use int index and columns name so that the order is correctly sorted.
    df_corr.index = [i for i in range(1, len(df_corr) + 1)]
    df_corr.columns = [i for i in range(1, len(df_corr) + 1)]
    
    links = df_corr.stack().reset_index()
    links.columns = ['roi1', 'roi2', 'corr']
    
    # use same threshold in the harmonic analysis:
    links_filtered=links.loc[(links['corr'] > 0.07) & (links['roi1'] != links['roi2'])]
 
    # Build your graph
    G=nx.from_pandas_edgelist(links_filtered, 'roi1', 'roi2')

    ## clustering coefficient:
    clustering = nx.clustering(G)
    clustering_list = [clustering[key] for key in sorted(clustering.keys())]
    all_clustering.append(clustering_list)
    
    ## betweenness:
    betweeness = nx.betweenness_centrality(G)
    betweeness_list = [betweeness[key] for key in sorted(betweeness.keys())]
    all_betweenness.append(betweeness_list)
    
    ## degree centrality:
    degree = nx.degree_centrality(G)
    degree_list = [degree[key] for key in sorted(degree.keys())]
    all_degree.append(degree_list)

    ## modularity:
    modularity = nx_comm.modularity(G, nx_comm.label_propagation_communities(G))
    all_modularity.append(modularity)
    
    ## eigenvector:
    eigenvector = nx.eigenvector_centrality(G)
    eigenvector_list = [eigenvector[key] for key in sorted(eigenvector.keys())]
    all_eigenvector.append(eigenvector_list)
    
    ## closeness:
    closeness = nx.closeness_centrality(G)
    closeness_list = [closeness[key] for key in sorted(closeness.keys())]
    all_closeness.append(closeness_list)
    
    # break
    


/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix1/corr_matrix_power264_sub-002_S_0295-ses-2011-06-02_07_56_36.0.txt
/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix1/corr_matrix_power264_sub-002_S_0295-ses-2012-05-10_15_42_37.0.txt
/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix1/corr_matrix_power264_sub-002_S_0413-ses-2011-06-16_12_18_49.0.txt
/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix1/corr_matrix_power264_sub-002_S_0413-ses-2012-05-15_16_38_28.0.txt
/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix1/corr_matrix_power264_sub-002_S_0413-ses-2013-05-10_07_08_02.0.txt
/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix1/corr_matrix_power264_sub-002_S_0685-ses-2011-07-08_07_04_27.0.txt
/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix1/corr_matrix_power264_sub-002_S_0729-ses-2011-08-16_15_07_45.0.txt
/home/xin/Downloads/BrainImaging_UNC/out05_adni_corr_matrix1/corr_matrix_power264_sub-002_S_1155-ses-2011-12-22

In [7]:

if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
for res, fname in [[all_clustering, 'clustering'], [all_modularity, 'modularity'], 
                   [all_betweenness, 'betweenness'], [all_eigenvector, 'eigenvector'],
                   [all_degree, 'degree'], [all_closeness, 'closeness']]:
    
    df = pd.DataFrame(res)
    output_clustering = pd.concat([output_file, df], axis = 1)
    output_clustering.to_csv(output_dir + '/' + fname + '.csv')
    